In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sqlite3 as sql
import os
import re

In [2]:
# Look for all file directories in the folder
directory = "./Jan_2010_Oct_2021/"
all_file_lst = []
for root, subdirectories, files in os.walk(directory):
    for file in files:
        all_file_lst.append(os.path.join(root, file))
# Only use csv file with "-street" in the name
needed_file_lst = [i for i in all_file_lst if "-outcome" in i]
needed_file_lst

['./Jan_2010_Oct_2021/2012-01\\2012-01-avon-and-somerset-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-bedfordshire-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-cambridgeshire-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-cheshire-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-city-of-london-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-cleveland-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-cumbria-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-derbyshire-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-devon-and-cornwall-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-dorset-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-durham-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-dyfed-powys-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-essex-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-gloucestershire-outcomes.csv',
 './Jan_2010_Oct_2021/2012-01\\2012-01-greater-manchester-outcomes.csv',
 './J

In [3]:
# Look for names of the sections per file using regular expression
section_lst = []
for i in needed_file_lst:
    section_name = re.search('20[0-9][0-9]-[0-9][0-9]-(.+?)-outcomes.csv', i)
    if section_name:
        found = section_name.group(1)
        section_lst.append(found)
section_lst

['avon-and-somerset',
 'bedfordshire',
 'cambridgeshire',
 'cheshire',
 'city-of-london',
 'cleveland',
 'cumbria',
 'derbyshire',
 'devon-and-cornwall',
 'dorset',
 'durham',
 'dyfed-powys',
 'essex',
 'gloucestershire',
 'greater-manchester',
 'hampshire',
 'hertfordshire',
 'humberside',
 'kent',
 'lancashire',
 'leicestershire',
 'lincolnshire',
 'merseyside',
 'metropolitan',
 'norfolk',
 'north-wales',
 'north-yorkshire',
 'northamptonshire',
 'northumbria',
 'nottinghamshire',
 'south-wales',
 'south-yorkshire',
 'staffordshire',
 'suffolk',
 'surrey',
 'sussex',
 'thames-valley',
 'warwickshire',
 'west-mercia',
 'west-midlands',
 'west-yorkshire',
 'wiltshire',
 'avon-and-somerset',
 'bedfordshire',
 'cambridgeshire',
 'cheshire',
 'city-of-london',
 'cleveland',
 'cumbria',
 'derbyshire',
 'devon-and-cornwall',
 'dorset',
 'durham',
 'dyfed-powys',
 'essex',
 'gloucestershire',
 'greater-manchester',
 'hampshire',
 'hertfordshire',
 'humberside',
 'kent',
 'lancashire',
 'lei

In [4]:
# Can be used in the future when we want to look for certain section
unique_section_lst = sorted(list(set(section_lst)))
unique_section_lst

['avon-and-somerset',
 'bedfordshire',
 'cambridgeshire',
 'cheshire',
 'city-of-london',
 'cleveland',
 'cumbria',
 'derbyshire',
 'devon-and-cornwall',
 'dorset',
 'durham',
 'dyfed-powys',
 'essex',
 'gloucestershire',
 'greater-manchester',
 'gwent',
 'hampshire',
 'hertfordshire',
 'humberside',
 'kent',
 'lancashire',
 'leicestershire',
 'lincolnshire',
 'merseyside',
 'metropolitan',
 'norfolk',
 'north-wales',
 'north-yorkshire',
 'northamptonshire',
 'northumbria',
 'nottinghamshire',
 'south-wales',
 'south-yorkshire',
 'staffordshire',
 'suffolk',
 'surrey',
 'sussex',
 'thames-valley',
 'warwickshire',
 'west-mercia',
 'west-midlands',
 'west-yorkshire',
 'wiltshire']

In [5]:
# # Create a list consisting of all the file names using all combinations of year, month and section

# # First all sub lists are defined

# # BTP is temporarily ignored
# all_section = ['Avon and Somerset', 'Bedfordshire', 'Cambridgeshire', 'Cheshire', 'City of London', 
#                'Cleveland', 'Cumbria', 'Derbyshire', 'Devon and Cornwall', 'Dorset', 'Durham', 'Dyfed-Powys', 
#                'Essex', 'Gloucestershire', 'Greater Manchester', 'Gwent', 'Hampshire', 'Hertfordshire', 
#                'Humberside', 'Kent', 'Lancashire', 'Leicestershire', 'Lincolnshire', 'Merseyside', 'Metropolitan', 
#                'Norfolk', 'North Wales', 'North Yorkshire', 'Northamptonshire', 'Northumbria', 'Nottinghamshire', 
#                'South Wales', 'South Yorkshire', 'Staffordshire', 'Suffolk', 'Surrey', 'Sussex', 'Thames Valley', 
#                'Warwickshire', 'West Mercia', 'West Midlands', 'West Yorkshire', 'Wiltshire']
# year_lst = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
# month_lst = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# # Concat year and month to form a list consisting of all possible months
# time_lst = []
# for i in year_lst:
#     for j in month_lst:
#         time_lst.append(i + '-' + j)
# del time_lst[-2: ]
# time_lst = time_lst[11: ]

# # Convert the format of all_section to suit file name
# street_lst = [i.lower().replace(' ', '-') for i in all_section]

# # Convert the above 2 lists to the file list
# file_lst = []
# for i in time_lst:
#     for j in street_lst:
#         file_lst.append(i + "/" + i + "-" + j)
# file_lst

In [6]:
# Create a new sql database file
conn = sql.connect('crime.db')
# It will loop over every file inside Jan_2010_Oct_2021 folder
for i in tqdm(range(len(needed_file_lst))):
    df = pd.read_csv(needed_file_lst[i])
    # Remove all data without geographical data
    df = df[df['Location'] != 'No location']
    # Create a column with the name of section (For easy searching when using SQL in the future)
    df['Section'] = section_lst[i]
    # Create two columns with year and month respectively (For easy searching and aggregation when using SQL in the future)
    time_df = df['Month'].str.split("-", n = 1, expand = True)
    df['Year'] = time_df[0]
    df['Month'] = time_df[1]
    # Convert the datatype of some attributes
    df = df.astype({"Year": int, "Month": int, "Longitude": "float64", "Latitude": "float64"})
    # Rename some columns
    df = df.rename(columns = {'Crime ID': 'Crime_ID', 'Reported by': 'Reported_by', 'Falls within': 'Falls_within', 
                          'LSOA code': 'LSOA_code', 'LSOA name': 'LSOA_name', 'Outcome type': 'Outcome_type'})
    # Append the dataframe into the SQL database
    df.to_sql('outcome_crime', conn, if_exists = 'append', index = False)

100%|██████████████████████████████████████████████████████████████████████████████| 5036/5036 [08:38<00:00,  9.72it/s]
